In [1]:
import cudf
import numpy as np
import pandas as pd
import math
from cuml.linear_model import LogisticRegression
from cuml.svm import SVC
from cuml import KMeans

%matplotlib inline
%load_ext autotime

# Read data

In [2]:
features = [
    ('srcip', 'str')
    , ('sport', 'str')
    , ('dstip', 'str')
    , ('dsport', 'str')
    , ('proto', 'str')
    , ('state', 'str')
    , ('dur', 'float64')
    , ('sbytes', 'int64')
    , ('dbytes', 'int64')
    , ('sttl', 'int64')
    , ('dttl', 'int64')
    , ('sloss', 'int64')
    , ('dloss', 'int64')
    , ('service', 'str')
    , ('sload', 'float64')
    , ('dload', 'float64')
    , ('spkts', 'int64')
    , ('dpkts', 'int64')
    , ('swin', 'int64')
    , ('dwin', 'int64')
    , ('stcpb', 'int64')
    , ('dtcpb', 'int64')
    , ('smeansz', 'int64')
    , ('dmeansz', 'int64')
    , ('trans_depth', 'int64')
    , ('res_bdy_len', 'int64')
    , ('sjit', 'float64')
    , ('djit', 'float64')
    , ('stime', 'int64')
    , ('ltime', 'int64')
    , ('sintpkt', 'float64')
    , ('dintpkt', 'float64')
    , ('tcprtt', 'float64')
    , ('synack', 'float64')
    , ('ackdat', 'float64')
    , ('is_sm_ips_ports', 'int64')
    , ('ct_state_ttl', 'int64')
    , ('ct_flw_http_mthd', 'int64')
    , ('is_ftp_login', 'int64')
    , ('ct_ftp_cmd', 'int64')
    , ('ct_srv_src', 'int64')
    , ('ct_srv_dst', 'int64')
    , ('ct_dst_ltm', 'int64')
    , ('ct_src_ ltm', 'int64')
    , ('ct_src_dport_ltm', 'int64')
    , ('ct_dst_sport_ltm', 'int64')
    , ('ct_dst_src_ltm', 'int64')
    , ('attack_cat', 'str')
    , ('label', 'int64')
]

names  = [e[0] for e in features]
dtypes = [e[1] for e in features]

temp = []
temp.append(cudf.read_csv('../data/UNSW-NB15_1.csv', names=names, dtype=dtypes))
temp.append(cudf.read_csv('../data/UNSW-NB15_2.csv', names=names, dtype=dtypes))
temp.append(cudf.read_csv('../data/UNSW-NB15_3.csv', names=names, dtype=dtypes))
temp.append(cudf.read_csv('../data/UNSW-NB15_4.csv', names=names, dtype=dtypes))

df = cudf.concat(temp)

for data in temp:
    del data

temp = []

time: 1.55 s


In [2]:
training_df = cudf.read_csv('../data/features/UNSW_NB15_training-set.csv')
testing_df  = cudf.read_csv('../data/features/UNSW_NB15_testing-set.csv')

### ADD IDENTIFIER
training_df['test'] = 0
testing_df['test']  = 1

# training_df = training_df.drop('id')
# testing_df  = testing_df.drop('id')

df = cudf.concat([training_df, testing_df])
df = df.drop(['id']).reset_index().rename({'index': 'id'})

df.head()

# ### ENCODE QUANTILES
# testing_binned = encode_quantiles(testing, quantiles)

# for col in carry_over_cols:
#     testing_binned[col] = testing[col]

# testing_binned = testing_binned.reset_index().rename({'index': 'id'})

# ### ENCODE CATEGORICAL
# cols_to_encode = ['proto', 'service', 'state']

# for col in cols_to_encode:
#     le = encoders[col]
#     testing_binned[col] = testing_binned[col].astype('category')
#     testing_binned[col + '_enc'] = le.transform(testing_binned[col])
    
#     encoders[col] = le
    
# cols_reordered = [c for c in testing_binned.columns if c not in cols_to_encode + ['label']] + ['label']
# testing_binned = testing_binned[cols_reordered]

# ### ENCODE ATTACK CAT
# testing_binned = testing_binned.merge(attack_categories, on='attack_cat')
# testing_binned = testing_binned.drop('attack_cat')
# testing_binned = testing_binned.reset_index().drop('id').rename({'index': 'id'})

# del testing     ### RELEASE THE MEMORY
# testing_binned.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label,test
0,0,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,0,0,0,1,1,0,Normal,0,0
1,1,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,2,0,0,0,1,6,0,Normal,0,0
2,2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,3,0,0,0,2,6,0,Normal,0,0
3,3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,3,1,1,0,2,1,0,Normal,0,0
4,4,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,40,0,0,0,2,39,0,Normal,0,0


time: 1.27 s


In [3]:
to_keep = [
      'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes'
    , 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss'
    , 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat'
    , 'ct_srv_src', 'ct_state_ttl', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm'
    , 'ct_srv_dst', 'attack_cat', 'label', 'test']

df = df[to_keep].reset_index(drop=True).reset_index()
df = df.rename({'index': 'id'})
df['id'] = df['id'].astype('int32')
df['attack_cat'] = df['attack_cat'].fillna('Normal')
df['attack_cat'] = df['attack_cat'].str.replace(' ', '')

time: 26.6 ms


In [4]:
df.describe()

,id,dur,spkts,dpkts,sbytes,dbytes,sttl,dttl,sload,dload,...,synack,ackdat,ct_srv_src,ct_state_ttl,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_srv_dst,label,test
count,257673.000000,257673.000000,257673.000000,257673.000000,2.576730e+05,2.576730e+05,257673.000000,257673.000000,2.576730e+05,2.576730e+05,...,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000,257673.000000
mean,128836.000000,1.246715,19.777144,18.514703,8.572952e+03,1.438729e+04,180.000931,84.754957,7.060869e+07,6.582143e+05,...,0.023652,0.022386,9.383176,1.324978,5.238271,4.032677,8.322964,9.121049,0.639077,0.319521
std,74383.932294,5.974305,135.947152,111.985965,1.737739e+05,1.461993e+05,102.488268,112.762131,1.857313e+08,2.412372e+06,...,0.053856,0.045771,10.829706,0.992300,8.160822,5.831515,11.120754,10.874752,0.480269,0.466292
min,0.000000,0.000000,1.000000,0.000000,2.400000e+01,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,64418.000000,0.000008,2.000000,0.000000,1.140000e+02,0.000000e+00,62.000000,0.000000,1.231800e+04,0.000000e+00,...,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000
50%,128836.000000,0.004285,4.000000,2.000000,5.280000e+02,1.780000e+02,254.000000,29.000000,7.439423e+05,1.747441e+03,...,0.000000,0.000000,5.000000,1.000000,1.000000,1.000000,3.000000,4.000000,1.000000,0.000000
75%,193254.000000,0.685777,12.000000,10.000000,1.362000e+03,1.064000e+03,254.000000,252.000000,8.000000e+07,2.210538e+04,...,0.036842,0.044665,12.000000,2.000000,4.000000,3.000000,8.000000,11.000000,1.000000,1.000000
max,257672.000000,59.999989,10646.000000,11018.000000,1.435577e+07,1.465753e+07,255.000000,254.000000,5.988000e+09,2.242273e+07,...,3.226788,2.928778,63.000000,6.000000,59.000000,46.000000,65.000000,62.000000,1.000000,1.000000


time: 330 ms


# Encode variables

## Continuous to bins

In [5]:
bin_num = 10

quantiles = cudf.DataFrame()

for i in range(1, bin_num):
    quant = i / 10
    print(f'Quantile: {quant}')
    
    quantiles['q' + str(i)] = df.quantile(q=quant)

quantiles = quantiles.dropna().reset_index()
quantiles_list = quantiles.to_pandas().to_dict('records')

Quantile: 0.1
Quantile: 0.2
Quantile: 0.3
Quantile: 0.4
Quantile: 0.5
Quantile: 0.6
Quantile: 0.7
Quantile: 0.8
Quantile: 0.9
time: 2.43 s


In [6]:
quantiles = []

for q in quantiles_list:
    if q['index'] not in ['id', 'label', 'test']:
        elements = list(q.items())
        quants = sorted(list(set([0.0] + [e[1] for e in elements[1:]])))
        quantiles.append((elements[0][1], quants))

time: 1.57 ms


In [7]:
def encode_quantiles(df, quantiles):
    temp_df = cudf.DataFrame()
    
    for q in quantiles:
        col_name = q[0]
        bins = q[1]

        if df[col_name].dtype == 'int64':
            bins = [int(e) for e in bins]

        temp_df[col_name + '_bin'] = df[col_name].digitize(np.array(bins))
    return temp_df

carry_over_cols = ['proto', 'service', 'state', 'attack_cat', 'label', 'test']
df_binned = encode_quantiles(df, quantiles)

for col in carry_over_cols:
    df_binned[col] = df[col]

df_binned = df_binned.reset_index().rename({'index': 'id'})
# del df
df_binned.head()

,id,dur_bin,spkts_bin,dpkts_bin,sbytes_bin,dbytes_bin,sttl_bin,dttl_bin,sload_bin,dload_bin,...,ct_src_dport_ltm_bin,ct_dst_sport_ltm_bin,ct_dst_src_ltm_bin,ct_srv_dst_bin,proto,service,state,attack_cat,label,test
0,0,6,3,2,4,1,3,3,3,3,...,2,2,2,2,tcp,-,FIN,Normal,0,0
1,1,8,4,6,5,6,3,3,2,5,...,2,2,3,6,tcp,-,FIN,Normal,0,0
2,2,10,3,5,4,6,3,3,1,4,...,2,2,4,6,tcp,-,FIN,Normal,0,0
3,3,10,4,4,5,4,3,3,1,2,...,2,2,4,2,tcp,ftp,FIN,Normal,0,0
4,4,7,4,3,5,2,4,3,2,3,...,3,2,8,9,tcp,-,FIN,Normal,0,0


time: 153 ms


## Categorical to index

In [8]:
import cuml

encoders = {}
cols_to_encode = ['proto', 'service', 'state']

for col in cols_to_encode:
    le = cuml.preprocessing.LabelEncoder()
    df_binned[col] = df_binned[col].astype('category')
    df_binned[col + '_enc'] = le.fit_transform(df_binned[col])
    
    encoders[col] = le
    
cols_reordered = [c for c in df_binned.columns if c not in cols_to_encode + ['label']] + ['label']
df_binned = df_binned[cols_reordered]
df_binned.columns

Index(['id', 'dur_bin', 'spkts_bin', 'dpkts_bin', 'sbytes_bin', 'dbytes_bin',
       'sttl_bin', 'dttl_bin', 'sload_bin', 'dload_bin', 'sloss_bin',
       'dloss_bin', 'sjit_bin', 'djit_bin', 'swin_bin', 'stcpb_bin',
       'dtcpb_bin', 'dwin_bin', 'tcprtt_bin', 'synack_bin', 'ackdat_bin',
       'ct_srv_src_bin', 'ct_state_ttl_bin', 'ct_src_dport_ltm_bin',
       'ct_dst_sport_ltm_bin', 'ct_dst_src_ltm_bin', 'ct_srv_dst_bin',
       'attack_cat', 'test', 'proto_enc', 'service_enc', 'state_enc', 'label'],
      dtype='object')

time: 193 ms


# Explode to COO format

In [9]:
### Attack_cat encoding
attack_cat = [
      (0, 'Normal')
    , (1, 'Reconnaissance')
    , (2, 'Shellcode')
    , (3, 'Analysis')
    , (4, 'Backdoor')
    , (5, 'Backdoors')
    , (6, 'DoS')
    , (7, 'Exploits')
    , (8, 'Generic')
    , (9, 'Fuzzers')
    , (10, 'Worms')
]

attack_cat = {
      'attack_id':  [e[0] for e in attack_cat]
    , 'attack_cat': [e[1] for e in attack_cat]
}

attack_categories = cudf.DataFrame(attack_cat)
df_binned = df_binned.merge(attack_categories, on='attack_cat')
df_binned = df_binned.drop('attack_cat')
df_binned.head()

,id,dur_bin,spkts_bin,dpkts_bin,sbytes_bin,dbytes_bin,sttl_bin,dttl_bin,sload_bin,dload_bin,...,ct_src_dport_ltm_bin,ct_dst_sport_ltm_bin,ct_dst_src_ltm_bin,ct_srv_dst_bin,test,proto_enc,service_enc,state_enc,label,attack_id
0,48416,2,2,1,1,1,4,1,9,1,...,2,2,5,5,0,119,0,5,1,9
1,48417,7,4,3,5,2,4,3,3,3,...,3,2,6,6,0,113,0,4,1,9
2,48418,10,6,6,9,6,4,3,6,3,...,2,2,2,2,0,113,5,4,1,9
3,48419,9,4,4,6,3,4,3,2,2,...,2,2,3,3,0,113,0,4,1,9
4,48420,8,4,3,6,2,4,3,3,2,...,2,2,3,3,0,113,0,4,1,9


time: 150 ms


In [10]:
training_df = df_binned.query('test == 0')
training_df = training_df.drop(['test', 'id']).reset_index().rename({'index': 'id'})

# training_df.head()
df_exploded = cudf.melt(training_df, id_vars=['id', 'label', 'attack_id']).sort_values('id').reset_index(drop=True)
df_exploded['variable'] = df_exploded['variable'].astype('str')
df_exploded['value'] = df_exploded['value'].astype('str')
df_exploded['feature'] = df_exploded['variable'] + '=' + df_exploded['value']
df_exploded.drop(['variable', 'value'])

feature_encoding = df_exploded['feature'].unique().reset_index()

time: 431 ms


In [11]:
len(df_binned), len(training_df)

(257673, 175341)

time: 1.98 ms


In [12]:
feature_encoding['index'] = feature_encoding['index'].astype('int16')
feature_encoding = feature_encoding.rename({'index': 'feature_enc'})
feature_encoding.head()

,feature_enc,feature
0,0,ackdat_bin=1
1,1,ackdat_bin=2
2,2,ackdat_bin=3
3,3,ackdat_bin=4
4,4,ackdat_bin=5


time: 23.9 ms


In [13]:
df_exploded = df_exploded.merge(feature_encoding, on='feature')[['id', 'feature_enc', 'label', 'attack_id']]
df_exploded.head()

,id,feature_enc,label,attack_id
0,564,270,0,0
1,565,85,0,0
2,565,266,0,0
3,565,66,0,0
4,565,228,0,0


time: 298 ms


# Finding frequent patterns

In [14]:
def mine_patterns(df_coo, df_binned, min_attack_rate=0.75, min_feature_count=200, max_iter=-1):
    def return_rank(frate, counts, rank):
        for i, (f, c) in enumerate(zip(frate, counts)):
            rank[i] = math.log(float(c)) * f
            
    features = df_coo['feature_enc'].unique().to_frame()
    df_coo = df_coo.merge(features, on='feature_enc')

    #### FIND FREQUENT ITEMS
    freq_items = df_coo.groupby(['feature_enc']).agg({'id': 'count', 'label': 'sum'}).reset_index()
    freq_items['attack_rate'] = freq_items['label'] / freq_items['id']

    freq_items = freq_items.apply_rows(
        return_rank
        , incols = {'label': 'counts', 'attack_rate': 'frate'}
        , outcols = {'rank': np.float64}
        , kwargs = {}
    ) 

    freq_items = freq_items.sort_values('rank', ascending=False)
    freq_items.head(5)

    freq_items = freq_items.query(f'attack_rate >= {min_attack_rate} and label > {min_feature_count}')

    if max_iter == -1:
        max_iter = len(freq_items)

    devices_checked = {}

    patterns = []
    stats    = []

    features_ordered = list(freq_items['feature_enc'].to_array())

    for i in range(max_iter):
        feature = features_ordered[i]

        ### get all the ids
        ids = df_coo.query('feature_enc == @feature')['id'].unique().to_frame()
        h = ids.hash_columns(['id']).sum()

#         print(f'Iter: {i}, Feature --> {feature}, devices analyzed --> {h in devices_checked}')

        if h not in devices_checked:    
            count_ids = len(ids)
            devices_checked[h] = 1

            ### OUTPUT PATTERN
            all_features = df_coo.merge(ids, on='id').groupby('feature_enc').agg({'label': 'count'}).query('label == @count_ids').reset_index()
            all_features['pattern_id'] = i

            patterns.append(all_features[['pattern_id', 'feature_enc']])

            ### OUTPUT STATS
            ids = ids.merge(df_binned, on='id')
            ids['pattern_id'] = i
            ids = ids.groupby('pattern_id').agg({'id': 'count', 'label': 'sum'})
            ids = ids.rename({'id': 'packet_count', 'label': 'attack_count'})
            ids['attack_rate'] = ids['attack_count'] / ids['packet_count']
            ids['feature_cnt'] = all_features['feature_enc'].count()
            ids = ids.reset_index()
            stats.append(ids)
            
    patterns = cudf.concat(patterns).merge(features, on='feature_enc').sort_values(by='pattern_id')
    stats = cudf.concat(stats)
    
    return patterns, stats
    
# patterns, stats = mine_patterns(df_exploded, training_df, min_attack_rate=.85, min_feature_count=200)
# stats

time: 3.63 ms


# Decode patterns (host/pandas)

In [60]:
def to_list(x):
    return list(x)

def ranges(bin_no, bins):
    if bin_no == len(bins):
        return '>{0:.0f}'.format(bins[-1])
    else:
        return '<{0:,.0f}, {1:,.0f})'.format(bins[bin_no-1], bins[bin_no])

quantile_bins = pd.DataFrame(quantiles, columns=['feat', 'bins'])
patterns_rec = patterns.merge(feature_encoding, on='feature_enc')

patterns_host = patterns_rec[['pattern_id', 'feature']].to_pandas()
patterns_host['feat']    = patterns_host.apply(lambda row: row['feature'].split('='), axis = 1)
patterns_host['bin']     = patterns_host.apply(lambda row: row['feat'][1], axis = 1)
patterns_host['feat']    = patterns_host.apply(lambda row: row['feat'][0][:-4], axis = 1)
patterns_host            = patterns_host.merge(quantile_bins, on=['feat'])
patterns_host['ranges']  = patterns_host.apply(lambda row: ranges(int(row['bin']), row['bins']), axis=1)
patterns_host['feature'] = patterns_host['feat'] + '=' + patterns_host['ranges']
patterns_host            = patterns_host[['pattern_id', 'feature']].sort_values(by='pattern_id')

patterns_host_agg               = patterns_host.groupby('feature').agg({'pattern_id': to_list}).reset_index()
patterns_host_agg['pattern_id'] = patterns_host_agg.apply(lambda row: ','.join([str(e) for e in row['pattern_id']]), axis = 1)#
patterns_host_agg               = patterns_host_agg.groupby('pattern_id').agg({'feature': to_list}).reset_index()

patterns_host_agg#.to_dict('records')

,pattern_id,feature
0,"0,1,2,4,7,8,9,10,15,16,17,24,25,29,31,32,33,34...","[dload=<0, 1,747)]"
1,"0,1,2,5,7,8,9,10,11,15,16,17,24,25,27,29,31,32...","[tcprtt=<0, 0)]"
2,"0,1,2,5,7,8,9,10,11,15,16,17,24,25,28,29,31,32...","[synack=<0, 0)]"
3,"0,1,2,5,7,8,9,10,11,15,16,17,24,25,29,31,32,33...","[ackdat=<0, 0), dloss=<0, 1), dtcpb=<0, 622,78..."
4,"0,1,2,6,7,8,9,10,15,16,17,24,25,29,31,32,33,34...","[dbytes=<0, 178)]"
5,"0,1,2,7,8,9,10,15,16,17,24,25,29,31,32,33,34,3...","[dttl=<0, 29)]"
6,"0,1,7,8,9,10,15,16,17,24,25,29,31,32,33,34,35,36","[djit=<0, 21), dpkts=<0, 2)]"
7,"0,5,7,8,9,10,11,15,16,17,24,25,29,31,32,33,34,...","[sloss=<0, 2)]"
8,"0,7,15,29,31,32,33,34,35,36","[swin=<0, 255)]"
9,"0,7,8,9,15,17,25,29,31,32,33,34,35,36","[stcpb=<0, 625,349,843)]"


time: 105 ms


# Testing

In [15]:
testing_binned = df_binned.query('test == 1')

time: 166 ms


## Pattern encoding

In [20]:
def encode_patterns(binned_df, patterns_to_encode):
    subset = binned_df[['id', 'label']]
    subset['pred'] = 0

    for i, pattern in enumerate(patterns_to_encode):
        q = ' and '.join([' == '.join(e) for e in pattern])
        temp = binned_df.query(q)['id'].to_frame()
        temp['enc'] = 1

        subset = subset.merge(temp, on=['id'], how='left')
        subset = subset.rename({'enc': 'col_' + str(i)})
        subset['col_' + str(i)] = subset['col_' + str(i)].fillna(0)
        subset['col_' + str(i)] = subset['col_' + str(i)].astype('float32')
        subset['pred'] = subset['pred'] + subset['col_'+str(i)]
        

    subset['pred'] = subset['pred'] > 0
    subset['pred'] = subset['pred'].astype('int8')
    
    return subset

patterns_rec['col_name'] = patterns_rec['feature'].str.split('=')[0]
patterns_rec['col_val']  = patterns_rec['feature'].str.split('=')[1]
patterns_rec = patterns_rec.sort_values(by='pattern_id').reset_index(drop=True)
patterns_expl = patterns_rec[['pattern_id', 'col_name', 'col_val']].to_pandas().to_records()

patterns_to_encode = []

curr = 0
temp = []
for i in patterns_expl:
    if curr == i[1]:
        temp.append((i[2], i[3]))
        
    else:
        patterns_to_encode.append(temp)
        temp = [(i[2], i[3])]
    curr = i[1]

encoded_testing = encode_patterns(testing_binned, patterns_to_encode)

#### Encoding training dataset
encoded_df = encode_patterns(training_df, patterns_to_encode)
X = encoded_df[['col_' + str(i) for i in range(len(patterns_to_encode))]]
y = encoded_df['label'].astype('float32')

[('ct_srv_dst_bin', '8')]

time: 30.2 ms


# End-2-end pattern mining

In [39]:
def calculate_metrics(df):
    ttl = df['id'].sum()
    accuracy = df.query('(label == 0 and pred == 0) or (label == 1 and pred == 1)')['id'].sum() / ttl
    
    fp = df.query('(label == 0 and pred == 1)')['id'].sum()
    tn = df.query('(label == 0 and pred == 0)')['id'].sum()
    fn = df.query('(label == 1 and pred == 0)')['id'].sum()
    tp = df.query('(label == 1 and pred == 1)')['id'].sum()
    
    fpr = fp / (fp + tn)
    fnr = fn / (fn + tp)
    
    return accuracy, (fpr + fnr) / 2

def encode_patterns(binned_df, patterns_to_encode):
    subset = binned_df[['id', 'label']]
    subset['pred'] = 0

    for i, pattern in enumerate(patterns_to_encode):
        q = ' and '.join([' == '.join(e) for e in pattern])
        temp = binned_df.query(q)['id'].to_frame()
        temp['enc'] = 1

        subset = subset.merge(temp, on=['id'], how='left')
        subset = subset.rename({'enc': 'col_' + str(i)})
        subset['col_' + str(i)] = subset['col_' + str(i)].fillna(0)
        subset['col_' + str(i)] = subset['col_' + str(i)].astype('float32')
        subset['pred'] = subset['pred'] + subset['col_'+str(i)]
        

    subset['pred'] = subset['pred'] > 0
    subset['pred'] = subset['pred'].astype('int8')
    
    return subset

def e2e_pattern_mining(label, df, training_df, testing_df, feature_encoding, results, min_attack_rate=.5, min_feature_count=200):
    print('[{0}] Mining features...'.format(label))
    patterns, stats = mine_patterns(df, training_df, min_attack_rate=min_attack_rate, min_feature_count=min_feature_count)
    patterns['label'] = label
    stats['label'] = label
    
    print('[{0}] Encoding patterns...'.format(label))
    patterns_rec = patterns.merge(feature_encoding, on='feature_enc')
    
    patterns_rec['col_name'] = patterns_rec['feature'].str.split('=')[0]
    patterns_rec['col_val']  = patterns_rec['feature'].str.split('=')[1]
    patterns_rec = patterns_rec.sort_values(by='pattern_id').reset_index(drop=True)
    patterns_expl = patterns_rec[['pattern_id', 'col_name', 'col_val']].to_pandas().to_records()

    patterns_to_encode = []

    curr = 0
    temp = []
    for i in patterns_expl:
        if curr == i[1]:
            temp.append((i[2], i[3]))

        else:
            patterns_to_encode.append(temp)
            temp = [(i[2], i[3])]
        curr = i[1]

    encoded_testing = encode_patterns(testing_df, patterns_to_encode)
    encoded_testing['id'] = encoded_testing['id'].astype('float32')

    #### Encoding training dataset
    encoded_df = encode_patterns(training_df, patterns_to_encode)
    X = encoded_df[['col_' + str(i) for i in range(len(patterns_to_encode))]]
    y = encoded_df['label'].astype('float32')
    
    #### Binned data
    X_binned = training_df[[col for col in training_df.columns if col not in ['id', 'label', 'attack_id']]]

    for col in X_binned:
        X_binned[col] = X_binned[col].astype('float32')

    y_binned = training_df['label'].astype('float32')
    
    X_testing_binned = testing_df[[col for col in testing_df.columns if col not in ['id', 'test', 'label', 'attack_id']]]
    for col in X_testing_binned:
        X_testing_binned[col] = X_testing_binned[col].astype('float32')
        
    y_testing_binned = testing_df[['id', 'label']]

   
    print('[{0}] Building models'.format(label))
    #####################################
    #### SIMPLE ENCODING
    #####################################
    pred = encoded_testing.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
    simple_results = calculate_metrics(pred)
    
    row_to_insert = {'label': label, 'model': 'simple', 'accuracy': simple_results[0], 'false-alarm-rate': simple_results[1]}
    simple_results_df = cudf.DataFrame(row_to_insert)
    
#     #####################################
#     #### LOGISTIC REGRESSION
#     #####################################
#     print('[{0}]\tLogistic Regression (patterns)'.format(label))
#     reg = LogisticRegression(fit_intercept=False, C=0.1)
#     reg.fit(X, y)

#     encoded_testing = encoded_testing.drop('pred')
#     encoded_testing['pred'] = reg.predict(encoded_testing[['col_' + str(i) for i in range(len(patterns_to_encode))]])
#     lr_conf = encoded_testing.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
#     lr_results = calculate_metrics(lr_conf)
    
#     row_to_insert = {'label': label, 'model': 'logistic regression (patterns)', 'accuracy': lr_results[0], 'false-alarm-rate': lr_results[1]}
#     lr_results_df = cudf.DataFrame(row_to_insert)
    
    #####################################
    #### SUPPORT VECTOR MACHINES
    #####################################
    print('[{0}]\tSupport Vector Machines (patterns)'.format(label))
    svc = SVC()
    svc.fit(X, y)

    encoded_testing = encoded_testing#.drop('pred')
    encoded_testing['pred'] = svc.predict(encoded_testing[['col_' + str(i) for i in range(len(patterns_to_encode))]])
    svc_conf = encoded_testing.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
    svc_results = calculate_metrics(svc_conf)
    
    row_to_insert = {'label': label, 'model': 'SVM_Classification (patterns)', 'accuracy': svc_results[0], 'false-alarm-rate': svc_results[1]}
    svc_results_df = cudf.DataFrame(row_to_insert)
    
#     #####################################
#     #### LOGISTIC REGRESSION (BINNED)
#     #####################################
#     print('[{0}]\tLogistic Regression (binned)'.format(label))
#     reg_binned = LogisticRegression(fit_intercept=True, C=0.1)
#     reg_binned.fit(X_binned, y_binned)

#     y_testing_binned['pred'] = reg_binned.predict(X_testing_binned)
#     lr_conf_binned = y_testing_binned.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
#     lr_results_binned = calculate_metrics(lr_conf_binned)
    
#     row_to_insert = {'label': label, 'model': 'logistic regression (binned)', 'accuracy': lr_results_binned[0], 'false-alarm-rate': lr_results_binned[1]}
#     lr_binned_results_df = cudf.DataFrame(row_to_insert)
    
    #####################################
    #### SUPPORT VECTOR MACHINES (BINNED)
    #####################################
    print('[{0}]\tSupport Vector Machines (binned)'.format(label))
    svc_binned = SVC()
    svc_binned.fit(X_binned, y_binned)

    y_testing_binned = y_testing_binned#.drop('pred')
    y_testing_binned['pred'] = svc_binned.predict(X_testing_binned)
    svc_pred_binned = y_testing_binned.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
    svc_results_binned = calculate_metrics(svc_pred_binned)
    
    row_to_insert = {'label': label, 'model': 'SVM_Classification (binned)', 'accuracy': svc_results_binned[0], 'false-alarm-rate': svc_results_binned[1]}
    svc_binned_results_df = cudf.DataFrame(row_to_insert)
    
#     results = cudf.concat([simple_results_df, lr_results_df, svc_results_df, lr_binned_results_df, svc_binned_results_df])
    results = cudf.concat([simple_results_df, svc_results_df, svc_binned_results_df])



    return patterns, stats, results

time: 10.7 ms


In [40]:
results  = cudf.DataFrame()
patterns = cudf.DataFrame()
stats    = cudf.DataFrame()

time: 835 µs


In [41]:
# Overall label

time: 984 µs


In [42]:
#### overall label
p, s, r = e2e_pattern_mining('0/1 Overall Label', df_exploded, training_df, testing_binned, feature_encoding, results, min_attack_rate=.75)
results  = cudf.concat([results,  r])
patterns = cudf.concat([patterns, p])
stats    = cudf.concat([stats,    s])

results

[0/1 Overall Label] Mining features...
[0/1 Overall Label] Encoding patterns...
[0/1 Overall Label] Building models
[0/1 Overall Label]	Support Vector Machines (patterns)
[0/1 Overall Label]	Support Vector Machines (binned)


,label,model,accuracy,false-alarm-rate
0,0/1 Overall Label,simple,0.618059,0.424946
0,0/1 Overall Label,SVM_Classification (patterns),0.815054,0.204926
0,0/1 Overall Label,SVM_Classification (binned),0.857771,0.156373


time: 31.4 s


In [43]:
def redefine_label(attack_id, new_label, attack_select):
    for i, ai in enumerate(attack_id):
        new_label[i] = 1 if ai == attack_select else 0
        
def 

for i, ac in list(zip(attack_cat['attack_id'], attack_cat['attack_cat']))[1:]:     
    training_df_new = training_df.apply_rows(
        redefine_label
        , incols  = ['attack_id']
        , outcols = {'new_label': np.int32}
        , kwargs  = {'attack_select': i}
    )

    testing_binned_new = testing_binned.apply_rows(
        redefine_label
        , incols  = ['attack_id']
        , outcols = {'new_label': np.int32}
        , kwargs  = {'attack_select': i}
    )

    training_df_new = training_df_new.rename('label_').rename({'new_label': 'label'})
    testing_binned_new = testing_binned_new.drop('label').rename({'new_label': 'label'})

    p, s, r = e2e_pattern_mining(ac + ' (full)', df_exploded, training_df_new, testing_binned_new, feature_encoding, results, min_attack_rate=.75)
    
    results  = cudf.concat([results,  r])
    patterns = cudf.concat([patterns, p])
    stats    = cudf.concat([stats,    s])
    
results

[Reconnaissance (full)] Mining features...
[Reconnaissance (full)] Encoding patterns...
[Reconnaissance (full)] Building models
[Reconnaissance (full)]	Support Vector Machines (patterns)
[Reconnaissance (full)]	Support Vector Machines (binned)
[Shellcode (full)] Mining features...
[Shellcode (full)] Encoding patterns...
[Shellcode (full)] Building models
[Shellcode (full)]	Support Vector Machines (patterns)
[Shellcode (full)]	Support Vector Machines (binned)
[Analysis (full)] Mining features...
[Analysis (full)] Encoding patterns...
[Analysis (full)] Building models
[Analysis (full)]	Support Vector Machines (patterns)
[Analysis (full)]	Support Vector Machines (binned)
[Backdoor (full)] Mining features...
[Backdoor (full)] Encoding patterns...
[Backdoor (full)] Building models
[Backdoor (full)]	Support Vector Machines (patterns)
[Backdoor (full)]	Support Vector Machines (binned)
[Backdoors (full)] Mining features...
[Backdoors (full)] Encoding patterns...
[Backdoors (full)] Building mod

/opt/conda/envs/rapids/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in int_scalars
  # This is added back by InteractiveShellApp.init_path()


RuntimeError: Exception occured! file=/conda/conda-bld/libcuml_1571339163826/work/cpp/src/svm/svc_impl.h line=86: Only binary classification is implemented at the moment
Obtained 64 stack frames
#0 in /opt/conda/envs/rapids/lib/python3.6/site-packages/cuml/common/../../../../libcuml++.so(_ZN8MLCommon9Exception16collectCallStackEv+0x3e) [0x7f51bb6f183e]
#1 in /opt/conda/envs/rapids/lib/python3.6/site-packages/cuml/common/../../../../libcuml++.so(_ZN8MLCommon9ExceptionC2ERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE+0x80) [0x7f51bb6f2350]
#2 in /opt/conda/envs/rapids/lib/python3.6/site-packages/cuml/common/../../../../libcuml++.so(_ZN2ML3SVM6svcFitIfEEvRKNS_10cumlHandleEPT_iiS6_RKNS0_12svmParameterERN8MLCommon6Matrix12KernelParamsERNS0_8svmModelIS5_EE+0x1741) [0x7f51bb98b6c1]
#3 in /opt/conda/envs/rapids/lib/python3.6/site-packages/cuml/svm/svm.cpython-36m-x86_64-linux-gnu.so(+0x21217) [0x7f51b3b52217]
#4 in /opt/conda/envs/rapids/lib/python3.6/site-packages/cuml/svm/svm.cpython

time: 1min 49s


In [31]:

# y.dtypes

time: 6.69 ms


## Results -- simple encoding

In [24]:
results = encoded_testing.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
results

,label,pred,id
0,0,0,19240
1,0,1,17760
2,1,0,5787
3,1,1,39545


time: 28.1 ms


In [25]:


calculate_metrics(results)

(0.7139994169946072, 0.30382908320832963)

time: 545 ms


## Logistic Regression model

In [32]:
reg = LogisticRegression(fit_intercept=False, C=0.1)
reg.fit(X, y)

encoded_testing = encoded_testing.drop('pred')
encoded_testing['pred'] = reg.predict(encoded_testing[['col_' + str(i) for i in range(len(patterns_to_encode))]])
results = encoded_testing.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
results
calculate_metrics(results)

(0.7139994169946072, 0.30382908320832963)

time: 772 ms


## Support Vector Machines

In [33]:
reg = SVC()
reg.fit(X, y)

encoded_testing = encoded_testing.drop('pred')
encoded_testing['pred'] = reg.predict(encoded_testing[['col_' + str(i) for i in range(len(patterns_to_encode))]])
results = encoded_testing.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
results
calculate_metrics(results)

(0.8366127386678327, 0.16515986201501953)

time: 5.77 s


# Binned features

In [44]:
# training_df.head()
X = training_df[[col for col in training_df.columns if col not in ['id', 'label', 'attack_id']]]

for col in X:
    X[col] = X[col].astype('float32')

y = training_df['label'].astype('float32')
X_testing = testing_binned[[col for col in testing_binned.columns if col not in ['id', 'test', 'label', 'attack_id']]]
y_testing = testing_binned[['id', 'label']]

reg = SVC()
reg.fit(X, y)

SVC(handle=<cuml.common.handle.Handle object at 0x7f6b004e16c0>, C=1, kernel='rbf', degree=3, gamma='auto', coef0=0.0, tol=0.001, cache_size=200.0, max_iter=-1, nochange_steps=1000, verbose=False)

time: 2.24 s


In [46]:
X.columns

Index(['dur_bin', 'spkts_bin', 'dpkts_bin', 'sbytes_bin', 'dbytes_bin',
       'sttl_bin', 'dttl_bin', 'sload_bin', 'dload_bin', 'sloss_bin',
       'dloss_bin', 'sjit_bin', 'djit_bin', 'swin_bin', 'stcpb_bin',
       'dtcpb_bin', 'dwin_bin', 'tcprtt_bin', 'synack_bin', 'ackdat_bin',
       'ct_srv_src_bin', 'ct_state_ttl_bin', 'ct_src_dport_ltm_bin',
       'ct_dst_sport_ltm_bin', 'ct_dst_src_ltm_bin', 'ct_srv_dst_bin',
       'proto_enc', 'service_enc', 'state_enc'],
      dtype='object')

time: 3.46 ms


In [52]:


for col in X_testing:
    X_testing[col] = X_testing[col].astype('float32')
    
# X_testing.columns
y_testing['pred'] = reg.predict(X_testing)
results = y_testing.groupby(['label', 'pred']).agg({'id': 'count'}).reset_index()
results
calculate_metrics(results)

(0.8577466841568284, 0.15639750215228906)

time: 139 ms


In [71]:
encoded_df.groupby('col_0').agg({'id': 'count', 'label': 'sum'})

,id,label
col_0,,
0.0,2259581,251932.0
1.0,280466,69351.0


time: 136 ms


In [104]:
sub = df_binned.query('ct_dst_sport_ltm_bin == 4')[['id', 'label']]
sub['pred'] = 1

sub['label'].sum(),sub['id'].count()
# sub

(22440, 22469)

time: 182 ms


In [98]:
fff = df_binned.merge(sub, on='id', how='left')#.head()
fff['pred'] = fff['pred'].fillna(0)

fff.groupby('pred').agg({'id': 'count', 'label_x': 'sum'})

,id,label_x
pred,,
0,2259581,59031
1,280466,262252


time: 69.4 ms
